In [1]:
import dataframe_image as dfi
import pandas as pd
import numpy as np

In [2]:
COUNTRY = "Yemen"

In [3]:
PATH_TO_DATA_FOLDER = "../Dataset time-series/output_data/" + COUNTRY + "/"

In [4]:
# Load the data.
df = pd.read_csv(PATH_TO_DATA_FOLDER + COUNTRY + "-day.csv", header = [0, 1], index_col = 0)
df.index.name = "Datetime"
df.index = pd.to_datetime(df.index)
freq = "D"
df.index.freq = freq

In [5]:
INDICATORS_TO_CONSIDER = ["FCS", "1 Month Anomaly (%) Rainfall", "3 Months Anomaly (%) Rainfall", "NDVI", "Cereals and tubers", "Exchange rate", "Fatality", "NDVI Anomaly", "rCSI", "Rainfall (mm)"]

In [6]:
# Consider only some indicators for the analysis with symbolic transfer entropy.
df = pd.concat([df.loc[:, (slice(None), feature)] for feature in INDICATORS_TO_CONSIDER], axis = 1).sort_index(axis = 1)

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled = scaler.fit_transform(df.values)
df_transform = pd.DataFrame(scaled, index = df.index, columns = df.columns)

## Correlations

### Nature indicator

In [8]:
select = df_transform.columns.get_level_values(1).isin(["NDVI", "NDVI Anomaly", "Rainfall (mm)", "3 Months Anomaly (%) Rainfall", "1 Month Anomaly (%) Rainfall"])
df_transform_nature = df_transform.loc[:, select]
df_transform_nature.head()

AdminStrata                        Abyan                                \
Indicator   1 Month Anomaly (%) Rainfall 3 Months Anomaly (%) Rainfall   
Datetime                                                                 
2018-08-22                     -1.314238                     -0.612732   
2018-08-23                     -1.338344                     -0.648345   
2018-08-24                     -1.360786                     -0.682703   
2018-08-25                     -1.381565                     -0.715805   
2018-08-26                     -1.400680                     -0.747652   

AdminStrata                                                              Aden  \
Indicator        NDVI NDVI Anomaly Rainfall (mm) 1 Month Anomaly (%) Rainfall   
Datetime                                                                        
2018-08-22  -1.382036    -0.892361      0.582397                    -0.951607   
2018-08-23  -1.378192    -0.900238      0.602560                    -0.971501   
2018-08-24  -1.374074    -0.908256      0.622723                    -0.989246   
2018-08-25  -1.369683    -0.916414      0.642886                    -1.004842   
2018-08-26  -1.365017    -0.924713      0.663049                    -1.018289   

AdminStrata                                                       \
Indicator   3 Months Anomaly (%) Rainfall      NDVI NDVI Anomaly   
Datetime                                                           
2018-08-22                       0.545490 -1.009102    -0.732608   
2018-08-23                       0.476178 -1.004211    -0.730864   
2018-08-24                       0.407308 -0.999166    -0.729090   
2018-08-25                       0.338880 -0.993968    -0.727285   
2018-08-26                       0.270895 -0.988616    -0.725450   

AdminStrata                ...                      Shabwah  \
Indicator   Rainfall (mm)  ... 1 Month Anomaly (%) Rainfall   
Datetime                   ...                                
2018-08-22      -0.743170  ...                    -1.332406   
2018-08-23      -0.720496  ...                    -1.361729   
2018-08-24      -0.697822  ...                    -1.388784   
2018-08-25      -0.675148  ...                    -1.413571   
2018-08-26      -0.652474  ...                    -1.436089   

AdminStrata                                                       \
Indicator   3 Months Anomaly (%) Rainfall      NDVI NDVI Anomaly   
Datetime                                                           
2018-08-22                      -0.518907 -1.735932    -0.851995   
2018-08-23                      -0.558543 -1.727584    -0.860700   
2018-08-24                      -0.596089 -1.718649    -0.869424   
2018-08-25                      -0.631545 -1.709128    -0.878169   
2018-08-26                      -0.664910 -1.699019    -0.886933   

AdminStrata                                      Taizz  \
Indicator   Rainfall (mm) 1 Month Anomaly (%) Rainfall   
Datetime                                                 
2018-08-22       0.433548                    -1.031774   
2018-08-23       0.464507                    -1.014098   
2018-08-24       0.495466                    -0.994305   
2018-08-25       0.526426                    -0.972395   
2018-08-26       0.557385                    -0.948367   

AdminStrata                                                                     
Indicator   3 Months Anomaly (%) Rainfall      NDVI NDVI Anomaly Rainfall (mm)  
Datetime                                                                        
2018-08-22                       0.977169 -0.237421    -0.419563      1.011644  
2018-08-23                       0.906129 -0.234354    -0.435289      1.063594  
2018-08-24                       0.835870 -0.230834    -0.451750      1.115544  
2018-08-25                       0.766393 -0.226864    -0.468944      1.167494  
2018-08-26                       0.697696 -0.222442    -0.486872      1.219444  

[5 rows x 100 columns]

In [9]:
corr_matrices_nature = list()

def correlation_matrices_nature(group):
    group.columns = group.columns.droplevel()
    mtrx = group.corr(method = "spearman").values
    corr_matrices_nature.append(mtrx)

df_transform_nature.groupby(level = 0, axis = 1).apply(correlation_matrices_nature);

In [10]:
# Mean.
CORR_nature_mean = pd.DataFrame(np.mean(np.stack(corr_matrices_nature, axis = 0), axis = 0), index = df_transform_nature.columns.droplevel().unique(), columns = df_transform_nature.columns.droplevel().unique())
corr = CORR_nature_mean.style.background_gradient(cmap = "coolwarm")
corr.export_png("./output_images/nature.png")
corr

Indicator,1 Month Anomaly (%) Rainfall,3 Months Anomaly (%) Rainfall,NDVI,NDVI Anomaly,Rainfall (mm)
Indicator,,,,,
1 Month Anomaly (%) Rainfall,1.000000,0.661091,0.318788,0.376140,0.203845
3 Months Anomaly (%) Rainfall,0.661091,1.000000,0.186227,0.413735,0.180265
NDVI,0.318788,0.186227,1.000000,0.771058,0.094741
NDVI Anomaly,0.376140,0.413735,0.771058,1.000000,0.105273
Rainfall (mm),0.203845,0.180265,0.094741,0.105273,1.000000


In [11]:
# Median.
CORR_nature_median = pd.DataFrame(np.median(np.stack(corr_matrices_nature, axis = 0), axis = 0), index = df_transform_nature.columns.droplevel().unique(), columns = df_transform_nature.columns.droplevel().unique())
CORR_nature_median.style.background_gradient(cmap = "coolwarm")

Indicator,1 Month Anomaly (%) Rainfall,3 Months Anomaly (%) Rainfall,NDVI,NDVI Anomaly,Rainfall (mm)
Indicator,,,,,
1 Month Anomaly (%) Rainfall,1.000000,0.647775,0.279437,0.341425,0.189780
3 Months Anomaly (%) Rainfall,0.647775,1.000000,0.135819,0.338473,0.165093
NDVI,0.279437,0.135819,1.000000,0.816205,0.144577
NDVI Anomaly,0.341425,0.338473,0.816205,1.000000,0.105529
Rainfall (mm),0.189780,0.165093,0.144577,0.105529,1.000000


In [12]:
# Variance.
CORR_nature_variance = pd.DataFrame(np.var(np.stack(corr_matrices_nature, axis = 0), axis = 0), index = df_transform_nature.columns.droplevel().unique(), columns = df_transform_nature.columns.droplevel().unique())
CORR_nature_variance.style.background_gradient(cmap = "coolwarm")

Indicator,1 Month Anomaly (%) Rainfall,3 Months Anomaly (%) Rainfall,NDVI,NDVI Anomaly,Rainfall (mm)
Indicator,,,,,
1 Month Anomaly (%) Rainfall,0.000000,0.013631,0.022021,0.046082,0.016780
3 Months Anomaly (%) Rainfall,0.013631,0.000000,0.063118,0.072662,0.048701
NDVI,0.022021,0.063118,0.000000,0.021136,0.050808
NDVI Anomaly,0.046082,0.072662,0.021136,0.000000,0.028719
Rainfall (mm),0.016780,0.048701,0.050808,0.028719,0.000000


### All indicators

In [13]:
corr_matrices = list()

def correlation_matrices(group):
    group.columns = group.columns.droplevel()
    mtrx = group.corr(method = "spearman").values
    corr_matrices.append(mtrx)

df_transform.groupby(level = 0, axis = 1).apply(correlation_matrices);

In [14]:
# Mean.
CORR_mean = pd.DataFrame(np.mean(np.stack(corr_matrices, axis = 0), axis = 0), index = df_transform.columns.droplevel().unique(), columns = df_transform.columns.droplevel().unique())
corr = CORR_mean.style.background_gradient(cmap = "coolwarm")
corr.export_png("./output_images/all.png")
corr

Indicator,1 Month Anomaly (%) Rainfall,3 Months Anomaly (%) Rainfall,Cereals and tubers,Exchange rate,FCS,Fatality,NDVI,NDVI Anomaly,Rainfall (mm),rCSI
Indicator,,,,,,,,,,
1 Month Anomaly (%) Rainfall,1.000000,0.661091,0.287159,0.373434,0.073301,-0.018462,0.318788,0.376140,0.203845,-0.092179
3 Months Anomaly (%) Rainfall,0.661091,1.000000,0.202382,0.267239,0.042599,0.040302,0.186227,0.413735,0.180265,-0.180728
Cereals and tubers,0.287159,0.202382,1.000000,0.499126,0.166868,-0.063713,0.229790,0.244262,0.041839,-0.013353
Exchange rate,0.373434,0.267239,0.499126,1.000000,0.192180,-0.057196,0.360971,0.293972,0.194754,0.036346
FCS,0.073301,0.042599,0.166868,0.192180,1.000000,0.065986,0.066857,0.064056,-0.041584,0.187032
Fatality,-0.018462,0.040302,-0.063713,-0.057196,0.065986,1.000000,-0.127209,-0.172948,-0.011933,0.021464
NDVI,0.318788,0.186227,0.229790,0.360971,0.066857,-0.127209,1.000000,0.771058,0.094741,0.012177
NDVI Anomaly,0.376140,0.413735,0.244262,0.293972,0.064056,-0.172948,0.771058,1.000000,0.105273,-0.092505
Rainfall (mm),0.203845,0.180265,0.041839,0.194754,-0.041584,-0.011933,0.094741,0.105273,1.000000,-0.117366


In [15]:
# Median.
CORR_median = pd.DataFrame(np.median(np.stack(corr_matrices, axis = 0), axis = 0), index = df_transform.columns.droplevel().unique(), columns = df_transform.columns.droplevel().unique())
CORR_median.style.background_gradient(cmap = "coolwarm")

Indicator,1 Month Anomaly (%) Rainfall,3 Months Anomaly (%) Rainfall,Cereals and tubers,Exchange rate,FCS,Fatality,NDVI,NDVI Anomaly,Rainfall (mm),rCSI
Indicator,,,,,,,,,,
1 Month Anomaly (%) Rainfall,1.000000,0.647775,0.361322,0.329668,0.014009,-0.070955,0.279437,0.341425,0.189780,-0.115074
3 Months Anomaly (%) Rainfall,0.647775,1.000000,0.277572,0.274744,0.032032,0.036471,0.135819,0.338473,0.165093,-0.205245
Cereals and tubers,0.361322,0.277572,1.000000,0.608493,0.215906,-0.099176,0.265550,0.294708,0.085790,0.023191
Exchange rate,0.329668,0.274744,0.608493,1.000000,0.216147,-0.136642,0.375667,0.296645,0.147799,0.142698
FCS,0.014009,0.032032,0.215906,0.216147,1.000000,0.093178,0.070117,0.063786,-0.080684,0.175826
Fatality,-0.070955,0.036471,-0.099176,-0.136642,0.093178,1.000000,-0.202325,-0.226255,-0.028251,0.013824
NDVI,0.279437,0.135819,0.265550,0.375667,0.070117,-0.202325,1.000000,0.816205,0.144577,0.015767
NDVI Anomaly,0.341425,0.338473,0.294708,0.296645,0.063786,-0.226255,0.816205,1.000000,0.105529,-0.086473
Rainfall (mm),0.189780,0.165093,0.085790,0.147799,-0.080684,-0.028251,0.144577,0.105529,1.000000,-0.074656


In [16]:
# Variance.
CORR_variance = pd.DataFrame(np.var(np.stack(corr_matrices, axis = 0), axis = 0), index = df_transform.columns.droplevel().unique(), columns = df_transform.columns.droplevel().unique())
CORR_variance.style.background_gradient(cmap = "coolwarm")

Indicator,1 Month Anomaly (%) Rainfall,3 Months Anomaly (%) Rainfall,Cereals and tubers,Exchange rate,FCS,Fatality,NDVI,NDVI Anomaly,Rainfall (mm),rCSI
Indicator,,,,,,,,,,
1 Month Anomaly (%) Rainfall,0.000000,0.013631,0.060215,0.021458,0.048321,0.075611,0.022021,0.046082,0.016780,0.037112
3 Months Anomaly (%) Rainfall,0.013631,0.000000,0.066182,0.028444,0.055836,0.089459,0.063118,0.072662,0.048701,0.040616
Cereals and tubers,0.060215,0.066182,0.000000,0.059701,0.035044,0.079605,0.056661,0.068367,0.051708,0.049033
Exchange rate,0.021458,0.028444,0.059701,0.000000,0.041150,0.096438,0.054151,0.054753,0.028819,0.050882
FCS,0.048321,0.055836,0.035044,0.041150,0.000000,0.035844,0.049445,0.073548,0.045362,0.031042
Fatality,0.075611,0.089459,0.079605,0.096438,0.035844,0.000000,0.129374,0.147872,0.040524,0.045952
NDVI,0.022021,0.063118,0.056661,0.054151,0.049445,0.129374,0.000000,0.021136,0.050808,0.039551
NDVI Anomaly,0.046082,0.072662,0.068367,0.054753,0.073548,0.147872,0.021136,0.000000,0.028719,0.053310
Rainfall (mm),0.016780,0.048701,0.051708,0.028819,0.045362,0.040524,0.050808,0.028719,0.000000,0.032278
